In [6]:
class RefPath:
    INITIAL = "#/info/x-clusters/"
    WORKER_NODES = "worker-nodes"
    PODS = "pods"
    CONTAINERS = "containers"
    INFO = "info"
    X_CLUSTERS = "x-clusters"
    X_LOCATION = "x-location"
    REF = "$ref"
    PATHS = "paths"
    
    def __init__(self, *arg):
        if len(arg) == 1:            
            paths = arg[0].split("#/info/x-clusters/")
            parts = paths[1].split("/")        
            self.cluster = parts[0]
            self.worker_node = parts[2]
            self.pod_name = parts[4]
            self.container_name = parts[6]
        else:
            self.cluster = arg[0]
            self.worker_node = arg[1]
            self.pod_name = arg[2]
            self.container_name = arg[3]    
        

    @property
    def full_path(self):
        return f"{RefPath.INITIAL}{self.cluster}/{RefPath.WORKER_NODES}/{self.worker_node}/{RefPath.PODS}/{self.pod_name}/{RefPath.CONTAINERS}/{self.container_name}"


class Method:
    def __init__(self, path_name, method_name, ref_path, load, schema_name, full_method):
        self.path_name = path_name
        self.method_name = method_name
        self.ref_path = ref_path      
        self.load = load
        self.schema_name = schema_name        
        self.full_method = full_method
        self.contribution = 0

    def __str__(self):
        return self.path_name + ": " + self.method_name + " (" + self.ref_path.full_path + ") " + str(self.load) + " <-> " + self.schema_name
    
class Component:
    def __init__(self, name, load, full_component, ref_path=None):
        self.name = name
        self.load = load
        self.full_component = full_component
        self.ref_path = ref_path
        
    def __str__(self):
        return self.name + " ---- " + str(self.load) + " (" + self.ref_path + ")"
    
import operator
class Group:
    def __init__(self, parent_load, sum_of_loads, components):        
        self.parent_load = parent_load        
        self.sum_of_loads = sum_of_loads
        self.components = components
        
    def get_contributed_components(self):
        components = []
        for component in self.components:
            component.contribution = (component.load / self.sum_of_loads) * self.parent_load
            components.append(component)        
        return sorted(components, key=operator.attrgetter("contribution"))


In [8]:
class Cluster(Component):
    def __init__(self, name, load, full_component, ref_path):        
        super().__init__(name, load, full_component, ref_path)
        self.is_cluster = True
        
    @property
    def worker_nodes(self):
        worker_nodes = []
        for wn in self.full_component["worker-nodes"].values():
            ref_path = self.ref_path + "/worker-nodes/" + wn["name"]
            worker_nodes.append(WorkerNode(wn["name"], wn["metrics"]["load"], wn, ref_path))
        return worker_nodes
        
class WorkerNode(Component):
    def __init__(self, pod_name, load, full_component, ref_path):
        super().__init__(pod_name, load, full_component, ref_path)
        self.is_worker_node = True
        
    @property
    def pods(self):
        pods = []
        for pod in self.full_component["pods"].values():
            ref_path = self.ref_path + "/pods/" + pod["name"]
            pods.append(Pod(pod["name"], pod["metrics"]["load"], pod, ref_path))        
        return pods
        
class Pod(Component):
    def __init__(self, pod_name, load, full_component, ref_path):
        super().__init__(pod_name, load, full_component, ref_path)
        self.is_pod = True
        self.contribution = 0
        
    @property
    def containers(self):
        containers = []
        for container_name, container in self.full_component["containers"].items():
            ref_path = self.ref_path + "/containers/" + container_name
            containers.append(Container(container_name, container["metrics"]["load"], container, ref_path))        
        return containers
        
class Container(Component):
    def __init__(self, pod_name, load, full_component, ref_path):
        super().__init__(pod_name, load, full_component, ref_path)
        self.is_container = True


class PodGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_pod_group = True            
        
    def __str__(self):
        return "PodGroup sum_of_pod_loads: " + str(sum_of_pod_loads)
    
class ContainerGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_container_group = True            
        
    def __str__(self):
        return "ContainerGroup sum_of_pod_loads: " + str(sum_of_pod_loads)
    
class MethodGroup(Group):
    def __init__(self, parent_load, sum_of_loads, components):
        super().__init__(parent_load, sum_of_loads, components)
        self.is_method_group = True            
        
    def __str__(self):
        return "ContainerGroup sum_of_pod_loads: " + str(sum_of_pod_loads)
        

In [20]:
def _gen_dict_extract(key, var):
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):                
                for result in _gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, (list)):
                for d in v:
                    for result in _gen_dict_extract(key, d):
                        yield result                        


def _get_schema_only(references):
    saved_schema = 'default'
    for reference in references:
        schema = reference.split("#/components/schemas/")        
        try:
            saved_schema = schema[1]
        except IndexError:
            pass
    
    return saved_schema


def _join_components(threshold, components):    
    joined_components = []
    remaining_components = []
    collective_contribution = 0
    for i in range(len(components)):
        component = components[i]
        collective_contribution += component.contribution
        
        if collective_contribution < threshold:
            joined_components.append(component)
        else:
            remaining_components.append(component)
        
    return joined_components, remaining_components


def _derive_components(single_data_obj):
    clusters = []
    cls = single_data_obj[RefPath.X_CLUSTERS]

    for cl in cls.values():
        ref_path = RefPath.INITIAL + cl["name"]
        clusters.append(Cluster(cl["name"], cl["metrics"]["load"], cl, ref_path))

    methods = []
    data_paths = single_data_obj[RefPath.PATHS]
    for path_name, path in data_paths.items():
        for method_name, method in path.items():
            ref_path = RefPath(method[RefPath.X_LOCATION][RefPath.REF])

            all_references = list(set(_gen_dict_extract('$ref', method)))
            schema_name = _get_schema_only(all_references)

            methods.append(
                Method(path_name, method_name, ref_path, method["x-metrics"]["load"], schema_name, method))
    return clusters, methods

In [1]:
# Monitor Scaling start from here
import json

with open("01data.json") as datafile:
    all_data = json.load(datafile)
    

with open("01config.json") as config_file:
    config_template = json.load(config_file)

In [88]:
cluster_template = config_template['info']['x-clusters']
name = next(iter(cluster_template))
cluster = Cluster(name, 0, cluster_template[name], '#/info/x-clusters/' + name)


paths_template = config_template['paths']

methods = []
for path_name, path in config_template['paths'].items():
    for method_name, method in path.items():
        
        ref_path = RefPath(method[RefPath.X_LOCATION][RefPath.REF])
        all_references = list(set(_gen_dict_extract('$ref', method)))
        schema_name = _get_schema_only(all_references)
        
        methods.append(Method(path_name, method_name, ref_path, 0, schema_name, method))
        
config_containers = {}
for wn in cluster.worker_nodes:    
    for pod in wn.pods:        
        for container in pod.containers:
            method = _get_method_by_ref(container.ref_path, methods)            
            if method:                
                config_container_ref = config_containers.setdefault(container.ref_path, {})
                config_container_ref_container = config_container_ref.setdefault('container', container)                
                config_container_ref_methods = config_container_ref.setdefault('methods', [])                
                config_container_ref_methods.append(method)


import copy
                
new_templates = []
for path, config_container in config_containers.items():
    copied_template = copy.deepcopy(config_template)
    ref_path = RefPath(path)
    container_obj = config_container['container']
    container = {container_obj.name: container_obj.full_component}
    
#     keeping only current worker-node
    wn_template = copied_template[ref_path.INFO][ref_path.X_CLUSTERS][ref_path.cluster][ref_path.WORKER_NODES]
    wn_template = {ref_path.worker_node: wn_template[ref_path.worker_node]}    
#     keeping only current pod
    pods_template = wn_template[ref_path.worker_node][ref_path.PODS]
    pods_template = {ref_path.pod_name: pods_template[ref_path.pod_name]}    
#     keeping only current container
    pods_template[ref_path.pod_name][ref_path.CONTAINERS] = container
    
    wn_template[ref_path.worker_node][ref_path.PODS] = pods_template    
    copied_template[ref_path.INFO][ref_path.X_CLUSTERS][ref_path.cluster][ref_path.WORKER_NODES] = wn_template
    new_templates.append(copied_template)

    
len(new_templates)

import json
with open("output.json", "w") as output_file:
    json.dump(new_templates[0], output_file, indent=2)
    

In [83]:
container_obj = config_container['container']
container = {container_obj.name: container_obj.full_component}

wn_template = copied_template[ref_path.INFO][ref_path.X_CLUSTERS][ref_path.cluster][ref_path.WORKER_NODES]
wn_template = {ref_path.worker_node: wn_template[ref_path.worker_node]}

wn_template[ref_path.worker_node][ref_path.PODS][ref_path.pod_name][ref_path.CONTAINERS] = container
copied_template[ref_path.INFO][ref_path.X_CLUSTERS][ref_path.cluster][ref_path.WORKER_NODES] = wn_template
copied_template

{'openapi': '3.0.0',
 'info': {'description': 'This is a sample server Petstore server.  You can find out more about     Swagger at [http://swagger.io](http://swagger.io) or on [irc.freenode.net, #swagger](http://swagger.io/irc/).      For this sample, you can use the api key `special-key` to test the authorization     filters.',
  'version': '1.0.0',
  'title': 'Swagger Petstore',
  'termsOfService': 'http://swagger.io/terms/',
  'contact': {'email': 'apiteam@swagger.io'},
  'license': {'name': 'Apache 2.0',
   'url': 'http://www.apache.org/licenses/LICENSE-2.0.html'},
  'x-clusters': {'cl1': {'name': 'cl1',
    'metrics': {'load': ''},
    'worker-nodes': {'wn2': {'name': 'wn2',
      'metrics': {'load': ''},
      'pods': {'pod1': {'name': 'pod1',
        'metrics': {'load': ''},
        'containers': {'c1': {'name': 'c1', 'metrics': {'load': ''}}}}}}}}}},
 'servers': [{'url': 'https://petstore.swagger.io',
   'description': 'Optional server description, e.g. Main (production) serve

In [71]:
a = {"no1": {"no2": "increase"}, "no3": {"no4": "decrease"}}
b = a["no1"]
a = b

a

{'no2': 'increase'}

In [30]:
def _get_method_by_ref(ref_path, methods):
    for method in methods:
        if method.ref_path.full_path == ref_path:
            return method
    return None

In [31]:
import copy

POD_LEVEL = "pod"
CL_LEVEL = "cluster"
WN_LEVEL = "worker-node"

copied_template = copy.deepcopy(config_template)

config_paths = config_template["paths"]
methods = []
for path_name, path in config_paths.items():
    for method_name, method in path.items():
        ref_path = RefPath(method["x-location"]["$ref"])
        
        all_references = list(set(_gen_dict_extract('$ref', method)))
        schema_name = _get_schema_only(all_references)

        methods.append(Method(path_name, method_name, ref_path, method["x-metrics"]["load"], schema_name, method))

method_schemas = {}
for method in methods:    
    method_schemas.setdefault(method.schema_name, {"pods": set(), "worker-nodes": set()})    
    method_schemas[method.schema_name]["pods"].add(method.ref_path.pod_name)
    method_schemas[method.schema_name]["worker-nodes"].add(method.ref_path.worker_node)
    

# it will remove the schema of stateless methods
if 'default' in method_schemas.keys():
    del method_schemas['default']

final_schema_levels = dict()
for schema_name, values in method_schemas.items():    
    if len(values["worker-nodes"]) > 1:
        final_schema_levels[schema_name] = CL_LEVEL
    elif len(values["pods"]) > 1:
        final_schema_levels[schema_name] = WN_LEVEL
    else:
        final_schema_levels[schema_name] = POD_LEVEL
        
config_schemas = copied_template['components']['schemas']
for schema_name, level in final_schema_levels.items():
    if schema_name in config_schemas.keys():
        config_schemas[schema_name]['x-storage-level'] = level

config_schemas

{'Order': {'x-storage-level': 'pod',
  'type': 'object',
  'properties': {'id': {'type': 'integer', 'format': 'int64'},
   'petId': {'type': 'integer', 'format': 'int64'},
   'quantity': {'type': 'integer', 'format': 'int32'},
   'shipDate': {'type': 'string', 'format': 'date-time'},
   'status': {'type': 'string',
    'description': 'Order Status',
    'enum': ['placed', 'approved', 'delivered']},
   'complete': {'type': 'boolean', 'default': False}},
  'xml': {'name': 'Order'}},
 'Category': {'x-storage-level': 'pod',
  'type': 'object',
  'properties': {'id': {'type': 'integer', 'format': 'int64'},
   'name': {'type': 'string'}},
  'xml': {'name': 'Category'}},
 'User': {'x-storage-level': 'worker-node',
  'type': 'object',
  'properties': {'id': {'type': 'integer', 'format': 'int64'},
   'username': {'type': 'string'},
   'firstName': {'type': 'string'},
   'lastName': {'type': 'string'},
   'email': {'type': 'string'},
   'password': {'type': 'string'},
   'phone': {'type': 'strin